In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import os.path as op
import glob
from dateutil.relativedelta import relativedelta
import datetime
import json
from pandas.io.json import json_normalize
from tqdm import tqdm_notebook as tqdm

In [2]:
#search for all json files in a specific folder
PTH = r'C:\Users\Liyonn\Desktop\ig-analysis'
FILES_JSON = glob.glob(op.join(PTH, '*.json')) 

In [3]:
FILES_JSON

['C:\\Users\\Liyonn\\Desktop\\ig-analysis\\kimkardashian.json',
 'C:\\Users\\Liyonn\\Desktop\\ig-analysis\\robkardashianofficial.json']

In [4]:
#select the precise json file
with open(FILES_JSON[0], encoding='utf-8', errors='ignore') as json_data:
         data = json.load(json_data, strict=False)

In [5]:
with open(FILES_JSON[1], encoding='utf-8', errors='ignore') as json_data2:
         data2 = json.load(json_data2, strict=False)

In [6]:
#loop through the data2 json file and make an array out of the posts
kim_posts=[]
for posts in data2['GraphImages']:
    kim_posts.append(posts)

In [7]:
#put the array into a dataframe
df_kim_images=pd.DataFrame(kim_posts)

In [8]:
#make an empty array. loop through the json data, and add the posts into the empty array
images=[]
for pics in data['GraphImages']:
    images.append(pics)

In [9]:
#make a dataframe from the array of posts
df_images=pd.DataFrame(images)

In [10]:
#combine the two dataframes
image_frames=[df_images, df_kim_images]
all_images=pd.concat(image_frames)

In [11]:
#remove the columns that aren't needed
df_images=df_images.drop(columns={'comments','gating_info', 'owner', 'tags', 'urls','thumbnail_src', 'media_preview','shortcode', 'thumbnail_resources', 'taken_at_timestamp'})

In [12]:
#remove the columns that aren't needed
all_images=all_images.drop(columns={'comments','gating_info', 'owner', 'tags', 'urls','thumbnail_src', 'media_preview','shortcode', 'thumbnail_resources', 'taken_at_timestamp'})

In [13]:
#rename the columns
all_images=all_images.rename(columns={'edge_media_to_caption': 'caption', 'edge_media_to_comment': 'comments', 'edge_media_preview_like':'likes', 'display_url':'url'})

In [14]:
#get the number of comments
comment_count=[]
for i in all_images.comments:
    comment_count.append(i['count'])

In [15]:
all_images['comments']=comment_count

In [16]:
#get the number of likes
likes_count=[]
for i in all_images.likes:
    likes_count.append(i['count'])

In [17]:
all_images['likes']=likes_count

In [18]:
all_images

,__typename,comments_disabled,dimensions,url,likes,caption,comments,id,is_video,username,video_view_count
0,GraphImage,False,"{'height': 1175, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,828548,{'edges': [{'node': {'text': 'What you’ve been...,5994,2276548965795267591,False,kimkardashian,NaN
1,GraphImage,False,"{'height': 1300, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,1093129,{'edges': [{'node': {'text': 'Shapewear is now...,5401,2276366116311797604,False,kimkardashian,NaN
2,GraphImage,False,"{'height': 1080, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,1606993,{'edges': [{'node': {'text': 'Found this pic o...,9673,2276326439940973637,False,kimkardashian,NaN
3,GraphImage,False,"{'height': 1350, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,1330759,{'edges': [{'node': {'text': 'Tomorrow shapewe...,7760,2275876008097949806,False,kimkardashian,NaN
4,GraphImage,False,"{'height': 1179, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,2060972,{'edges': [{'node': {'text': 'My hair is gonna...,11911,2275764997302972339,False,kimkardashian,NaN
...,...,...,...,...,...,...,...,...,...,...,...
179,GraphImage,False,"{'height': 1080, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,11383,{'edges': [{'node': {'text': '@arthurgeorge87 ...,319,2050973268052378958,False,robkardashianofficial,NaN
180,GraphImage,False,"{'height': 1080, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,4508,{'edges': [{'node': {'text': '@halfwaydead'}}]},50,2050967271556611271,False,robkardashianofficial,NaN
181,GraphImage,False,"{'height': 1245, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,5684,{'edges': [{'node': {'text': '@halfwaydead H W...,58,2050966323098648358,False,robkardashianofficial,NaN
182,GraphImage,False,"{'height': 1080, 'width': 1080}",https://scontent-waw1-1.cdninstagram.com/v/t51...,5881,{'edges': [{'node': {'text': 'HWD @halfwaydead...,127,2050965748093146048,False,robkardashianofficial,NaN


In [19]:
#create a 2D array. loop through every post and number it. one column of the array gets all the comments. the second..
# gets the number of the post
post_id=-1
comments=[[]for post in data['GraphImages']]
for post in data['GraphImages']:
    post_id+=1
    for i in post['comments']['data']:
        comments[0].append(i['text'])
        comments[1].append(post_id)
#kim_comments=[[]for post2 in data2['GraphImages']]
#for post2 in data2['GraphImages']:
 #   post_id+=1
  #  for i in post2['comments']['data']:
   #     comments[0].append(i['text'])
    #    comments[1].append(post_id)

In [20]:
all_comments=pd.DataFrame(columns=['comment','post_id'])

In [21]:
#make a dataframe from the array with comments
all_comments['comment']=comments[0]
all_comments['post_id']=comments[1]

In [22]:
all_comments

,comment,post_id
0,میمون,0
1,👏,0
2,The pieces together,0
3,Follow this MEME PAGE you won't regret!,0
4,😍😍😍,0
...,...,...
1959944,👍,189
1959945,Bomb 💖💖💖,189
1959946,😍😍😍😍❤️,189
1959947,Amazing 😍,189


In [23]:
pip install vaderSentiment

ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\Users\Liyonn\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _dep_map
    return self.__dep_map
  File "C:\Users\Liyonn\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Liyonn\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3003, in _parsed_pkg_info
    return self._pkg_info
  File "C:\Users\Liyonn\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Liyonn\Anaconda3\lib\site-packages\pip\_

Note: you may need to restart the kernel to use updated packages.


In [24]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [25]:
analyser = SentimentIntensityAnalyzer()

In [26]:
sentiment_comments=[]
for c in comments[0]:
    vs = analyser.polarity_scores(c)
    sentiment_comments.append(vs)

In [27]:
score=[]
for i in sentiment_comments:
    score.append(i['compound'])

In [28]:
positive=0
negative=0
for i in score:
    if i<negative:
        negative=i
    if i>positive:
        positive=i

In [29]:
positive

1.0